In [24]:
# Load packages

import sys
sys.path.append('/Users/ko389/Documents/Arctic_Water_Masses/Functions_general/')
import pandas as pd
import numpy as np
import xarray as xr
import gsw

In [25]:
data_path = '/Users/ko389/Documents/GitHub/Arctic_water_masses/Data/water_mass_data.nc'
water_mass_data = xr.open_dataset(data_path).to_dataframe()
data_path = '/Users/ko389/Documents/GitHub/Arctic_water_masses/Data/water_mass_data_stdev.nc'
water_mass_data_stdev = xr.open_dataset(data_path).to_dataframe()

In [26]:
# Add the time information from T, S, DO dataset

#Load Arctic T, S, DO data
arctic = xr.open_dataset("/Users/ko389/Documents/GitHub/Arctic_water_masses/Data/Arctic_T_S_DO_data.nc").to_dataframe()
def set_up_arctic_data(df):

    # rename & reorder columns
    df=df.rename(columns={'dissolved_oxygen':'oxygen','conservative_temperature':'conservative_temp'})
    column_order = ['conservative_temp', 'absolute_salinity', 'oxygen','latitude', 'longitude', 'depth', 'source','nprof', 'datetime','pressure']
    df = df [column_order]

    # remove labrador sea
    labrador_Sea = df[(df['latitude']<80)&(df['longitude']<-40)&(df['longitude']>-120)].index
    df = df.drop(labrador_Sea)

    # remove data with depth > 1000m
    df = df[df['depth']<1000]
    
    return df
arctic = set_up_arctic_data(arctic)


# Combine with water mass data
water_mass_data.reset_index(drop=True, inplace=True)
water_mass_data_stdev.reset_index(drop=True, inplace=True)
arctic.reset_index(drop=True, inplace=True)
#water_mass_data = water_mass_data.drop_duplicates(subset=['latitude', 'depth','conservative_temp', 'absolute_salinity'])
#arctic = pd.merge(water_mass_data, arctic, on=['latitude','depth','conservative_temp', 'absolute_salinity'], how='inner')
water_mass_data= pd.concat([water_mass_data, arctic['oxygen'],arctic['datetime'], arctic['source'], arctic['nprof']], axis=1)
water_mass_data_stdev = pd.concat([water_mass_data_stdev, arctic['oxygen'],arctic['datetime'], arctic['source'], arctic['nprof']], axis=1)

/Users/ko389/miniconda3/envs/hydro_env_local/lib/python3.12/site-packages/xarray/coding/times.py:254: RuntimeWarning: invalid value encountered in cast
  flat_num_dates_ns_int = (flat_num_dates * _NS_PER_TIME_DELTA[delta]).astype(


In [27]:
water_mass_data.drop(columns=['lon_sin', 'lon_cos', 'season', 'seasonal_sin'], inplace=True)
water_mass_data_stdev.drop(columns=['lon_sin', 'lon_cos', 'season', 'seasonal_sin'], inplace=True)

In [ ]:
# Remove salinity < 0.5 - we notice this after processing & compilation of T, S, DO data
water_mass_data = water_mass_data[water_mass_data['absolute_salinity']>=0.5]
water_mass_data_stdev = water_mass_data_stdev[water_mass_data_stdev['absolute_salinity']>=0.5]

In [ ]:
# rename columns ACCW to sPW and MAW to AW - some renaming of water masses since analysis
water_mass_data.rename(columns={'ACCW':'sPW', 'MAW':'AW'}, inplace=True)
water_mass_data_stdev.rename(columns={'ACCW':'sPW', 'MAW':'AW'}, inplace=True)

In [32]:
# Convert to xarray Dataset (without setting too many indices)
ds = xr.Dataset.from_dataframe(water_mass_data)
ds2 = xr.Dataset.from_dataframe(water_mass_data_stdev)

# Save to NetCDF
output_path = "/Users/ko389/Documents/GitHub/Arctic_water_masses/Data/WMA_fractions.nc"#_without_DO.nc"
ds.to_netcdf(output_path, engine="netcdf4")
output_path_stdev = "/Users/ko389/Documents/GitHub/Arctic_water_masses/Data/WMA_stdev.nc"#_without_DO.nc"
ds2.to_netcdf(output_path_stdev, engine="netcdf4")

In [42]:
# Save as CSV

water_mass_data.columns = [
    'Arctic_Surface_Water_[fraction]',
    'Modified_summer_Pacific_Water_[fraction]',
    'Summer_Pacific_Water_[fraction]',
    'Winter_Pacific_Water_[fraction]',
    'Norwegian_Current_Water_[fraction]',
    'Atlantic_Water_[fraction]',
    'Brine-enriched_Water_[fraction]',
    'Conservative_Temperature_[deg_C]',
    'Absolute_Salinity_[PSU]',
    'Latitude_[deg_N]',
    'Depth_[m]',
    'Longitude_[deg_E]',
    'Dissolved_Oxygen_[micro_mol_per_kg]',
    'Datetime_[UTC]',
    'Source',
    'Profile_Number'
]

water_mass_data.to_csv('/Users/ko389/Documents/GitHub/Arctic_water_masses/Data/WMA_fractions.csv', index=False, header=True)